This project was inspired by this university paper written back in 2015. I tried finding the datasets online but was unable to, so I decided to replicate the work done in this paper. 

http://uu.diva-portal.org/smash/get/diva2:846981/FULLTEXT01.pdf 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
prefix = "http://www.perverted-justice.com"
webpage_response = requests.get('http://www.perverted-justice.com/?archive=byUserVotes')

In [3]:
#stores the content of webpage we're scraping for links
webpage = webpage_response.content
soup = BeautifulSoup(webpage, "lxml") #lxml is faster than html.parser"

In [4]:
#stores all 'a' tags with attribute "id" equal to "pedoLink"
pedo_links = soup.findAll("a", {"id": "pedoLink"})
count = len(pedo_links)
print(count)

627


Bingo! That's the number of pedo links provided on website! Making progress.

In [5]:
links = []
# We take the href part of the a tag, then marry that to the prefix, which is the base part for each link
for a in pedo_links:
    links.append(prefix+a["href"])

In [6]:
#pedo_data:\n",
pedo_data = {}

metadata_list = []
# I want to pull all the span tags with attribute inText
# Then I want to pull the entire conversation 
for link in links:
    subpage = requests.get(link)
    subsoup = BeautifulSoup(subpage.content, "lxml")
    metadata= subsoup.findAll("span", {"class": "inText"}) #finds all the span tags that have class = inText
    str1 = ''.join(str(e) for e in metadata) #converts metadata into strings
    temp_list = re.compile('<span class="inText">(.*?)</span>', re.DOTALL).findall(str1) #pulls only the text between the span tags
    if len(temp_list) == 6:
        temp_list.insert(1, '0')
        print(temp_list)
    metadata_list.append(temp_list)

['Loren', '0', 'magic4isl2002', 'Poway, San Diego, California', 'Tiffanyjoy2cute', '13 year old', 'girl']
['Jacob', '0', 'jacob_cason2003', 'Nashville, Georgia', 'sassicassi93', '13 year old', 'girl']
['William', '0', 'welfare_isforwhitefolks', ', North Carolina', 'jewelykool', '14 year old', 'girl']
['Timothy', '0', 'baddboyysweetheart', 'Beaver Dam, Wisconsin', 'lil_suzie_qt', '13 year old', 'girl']
['Brian Richard Williams', '0', 'bmichigan69', 'Clarkston, Michigan', 'yayitskc95', '14 year old', 'girl']


In [7]:
df = pd.DataFrame(metadata_list,columns=['pedo_name','pedo_age','pedo_userid','city_state','victim_userid','victim_age','victim_gender'])
print(df.head())

                        pedo_name   pedo_age                pedo_userid  \
0                      Paul Short         34  fleet_captain_jaime_wolfe   
1               Michael J. Coffey         54                DavieWants2   
2                             Joe         28              vamale_692005   
3                   Kalonji Woods         21               kalowoodsman   
4  Keith Prather, Timothy Thompso  44 and 26             thenewperson62   

                            city_state   victim_userid   victim_age  \
0                    Chicago, Illinois      sadlilgrrl  13 year old   
1             Suffolk County, New York   SnapShotDeath  14 year old   
2                 Alexandria, Virginia    sweet_erin78  14 year old   
3                 Radford VA, Virginia            keri     underage   
4  Conroe / Willis / Montgomery, Texas  jackies2cool4u  13 year old   

  victim_gender  
0          girl  
1           boy  
2          girl  
3          girl  
4          girl  


In [8]:
new = df["city_state"].str.split(",",expand=True)

In [9]:
df["city"]= new[0] 
df["state"]= new[1] 

In [10]:
print(df.head())

                        pedo_name   pedo_age                pedo_userid  \
0                      Paul Short         34  fleet_captain_jaime_wolfe   
1               Michael J. Coffey         54                DavieWants2   
2                             Joe         28              vamale_692005   
3                   Kalonji Woods         21               kalowoodsman   
4  Keith Prather, Timothy Thompso  44 and 26             thenewperson62   

                            city_state   victim_userid   victim_age  \
0                    Chicago, Illinois      sadlilgrrl  13 year old   
1             Suffolk County, New York   SnapShotDeath  14 year old   
2                 Alexandria, Virginia    sweet_erin78  14 year old   
3                 Radford VA, Virginia            keri     underage   
4  Conroe / Willis / Montgomery, Texas  jackies2cool4u  13 year old   

  victim_gender                          city      state  
0          girl                       Chicago   Illinois  
1   

In [11]:
new2 = df["victim_age"].str.split(" ",expand=True)
df["child_age"] = new2[0]
print(df.head())

                        pedo_name   pedo_age                pedo_userid  \
0                      Paul Short         34  fleet_captain_jaime_wolfe   
1               Michael J. Coffey         54                DavieWants2   
2                             Joe         28              vamale_692005   
3                   Kalonji Woods         21               kalowoodsman   
4  Keith Prather, Timothy Thompso  44 and 26             thenewperson62   

                            city_state   victim_userid   victim_age  \
0                    Chicago, Illinois      sadlilgrrl  13 year old   
1             Suffolk County, New York   SnapShotDeath  14 year old   
2                 Alexandria, Virginia    sweet_erin78  14 year old   
3                 Radford VA, Virginia            keri     underage   
4  Conroe / Willis / Montgomery, Texas  jackies2cool4u  13 year old   

  victim_gender                          city      state child_age  
0          girl                       Chicago   Illin

In [12]:
df = df.drop(['city_state', 'victim_age'], axis=1) #axis=1 specifies this is applied to columns (column operation)

In [13]:
df.groupby('victim_gender').count()

,pedo_name,pedo_age,pedo_userid,victim_userid,city,state,child_age
victim_gender,,,,,,,
boy,52,52,52,52,52,52,52
girl,549,549,549,549,549,549,549


In [14]:
df.groupby('child_age').count()

,pedo_name,pedo_age,pedo_userid,victim_userid,victim_gender,city,state
child_age,,,,,,,
11,2,2,2,2,2,2,2
12,46,46,46,46,46,46,46
12/13,1,1,1,1,1,1,1
13,332,332,332,332,332,332,332
13/14,1,1,1,1,1,1,1
14,184,184,184,184,184,184,184
15,23,23,23,23,23,23,23
underage,12,12,12,12,12,12,12


In [15]:
df.groupby('pedo_age').count()

,pedo_name,pedo_userid,victim_userid,victim_gender,city,state,child_age
pedo_age,,,,,,,
"""28""",1,1,1,1,1,1,1
0,5,5,5,5,5,5,5
19,6,6,6,6,6,6,6
20,8,8,7,7,8,8,7
21,25,25,24,24,25,25,24
22,28,28,26,26,28,28,26
23,34,34,34,34,34,34,34
24,30,30,29,29,30,30,29
25,26,26,23,23,26,26,23


In [16]:
df['pedo_age'] = df.pedo_age.str.replace('"28"', '28')

In [17]:
df.groupby('pedo_age').count()

,pedo_name,pedo_userid,victim_userid,victim_gender,city,state,child_age
pedo_age,,,,,,,
0,5,5,5,5,5,5,5
19,6,6,6,6,6,6,6
20,8,8,7,7,8,8,7
21,25,25,24,24,25,25,24
22,28,28,26,26,28,28,26
23,34,34,34,34,34,34,34
24,30,30,29,29,30,30,29
25,26,26,23,23,26,26,23
26,38,38,38,38,38,38,38


In [ ]:
#Seperate age from string
#boys = 1 girls = 0
#distribution of pedo_age
#distribution of city/state

In [ ]:
#https://stackoverflow.com/questions/57484530/beautifulsoup-find-all-and-get-text for conversation part of website